In [2]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier

train = pd.read_csv("output/train_preprocessed.csv")


In [3]:
Y_train = train["y"]

In [12]:
X_train = train.loc[:,['buying', 'maint', 'doors', 'persons', 'lug_boot', 'safety']]

In [13]:
X_train.head()

,buying,maint,doors,persons,lug_boot,safety
0,1,2,3,2,1,1
1,1,3,3,5,1,2
2,4,3,5,2,1,2
3,3,3,3,5,3,2
4,3,3,3,2,2,3


In [14]:
y_train = Y_train.values.ravel()

In [15]:
RF = RandomForestClassifier(n_estimators = 300, random_state = 42)
RF.fit(X_train, y_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=300, n_jobs=1,
            oob_score=False, random_state=42, verbose=0, warm_start=False)

In [16]:
y_train_pred = RF.predict(X_train)

In [17]:
from sklearn.metrics import confusion_matrix 

In [18]:
cmat = confusion_matrix(y_train_pred, Y_train, labels=[1,2,3,4])
print(cmat)
(cmat[0,0]+cmat[1,1]+cmat[2,2]+cmat[3,3])/len(y_train_pred)

[[591   0   0   0]
 [  0 206   0   0]
 [  0   0  34   0]
 [  0   0   0  33]]


1.0

## おそらく過学習。GridSearchで過学習を避けるポイントをみつけないと